<a href="https://colab.research.google.com/github/Blackbadger313/fingerprintPattern-CNN/blob/main/Fingerprint_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!!pip install fingerprint-enhancer
!!pip install fingerprint-feature-extractor
!!pip install Pillow

In [ ]:
import os
import random
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import shutil
from tqdm import tqdm
from google.colab import drive
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import load_model
from PIL import Image

In [ ]:
#fetch data set then do extraction
!wget -N https://cainvas-static.s3.amazonaws.com/media/user_data/cainvas-admin/dataset_HFu5SVU.zip
!unzip -qo dataset_HFu5SVU.zip
dir = 'dataset'

In [ ]:
def list_picture_files(directory):
    picture_extensions = ['.png']
    file_names = []
    for file in os.listdir(directory):
        if os.path.isfile(os.path.join(directory, file)):
            file_extension = os.path.splitext(file)[1].lower()
            if file_extension in picture_extensions:
                file_name = file  # Get file name without extension
                file_names.append(file_name)
    return file_names

def list_txt_files(directory):
    picture_extensions = ['.txt']
    file_names = []
    for file in os.listdir(directory):
        if os.path.isfile(os.path.join(directory, file)):
            file_extension = os.path.splitext(file)[1].lower()
            if file_extension in picture_extensions:
                file_name = file  # Get file name without extension
                file_names.append(file_name)
    return file_names

# Replace 'directory_path' with the path to the directory you want to list the files from
dataset_folder = '/content/dataset/'
file_list = list_picture_files(dataset_folder)
txt_list = list_txt_files(dataset_folder)

In [ ]:
# Load the library
import fingerprint_enhancer

# Mount Google Drive
drive.mount('/content/drive')

output_folder = '/content/drive/MyDrive/EnhancedFingerprintImage/'  # Output folder path in Google Drive

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

with tqdm(total=len(txt_list), desc="Copying txt files", unit="file") as pbar:
    for file_name in txt_list:
        source_path = os.path.join(dataset_folder, file_name)
        destination_path = os.path.join(output_folder, file_name)
        shutil.copy2(source_path, destination_path)
        pbar.update(1)

with tqdm(total=len(file_list), desc="Processing picture files", unit="file") as pbar:
    for file_name in file_list:
        img = cv2.imread(os.path.join(dataset_folder, file_name), 0)  # Read input image
        out = fingerprint_enhancer.enhance_Fingerprint(img, resize=True)  # Enhance the fingerprint image
        output_path = os.path.join(output_folder, file_name)
        cv2.imwrite(output_path, out)
        origina_size = Image.open(output_path)
        resized_image = origina_size.resize((128, 128))
        resized_image.save(output_path)
        pbar.update(1)

print('Processing complete!')


In [ ]:
drive.mount('/content/drive')
output_folder = '/content/drive/MyDrive/EnhancedFingerprintImage/'

In [ ]:
labels = []
img_names = []
img_paths = []
gender = []
files = os.listdir(output_folder)

with tqdm(total=len(files), desc='Processing files') as pbar:
    for file in files:
        if file.endswith('.txt'):
            with open(os.path.join(output_folder, file), 'r') as t:
                content = t.readlines()
                gender.append(content[0].rsplit(' ')[1][0])
                img_name = content[2].rsplit(' ')[1][:-4] + '.png'
                img_paths.append(os.path.join(output_folder, img_name))
                img_names.append(img_name)
                labels.append(content[1].rsplit(' ')[1][0])

        pbar.update(1)

In [ ]:
#fill up data to data frame
print(labels)
print(img_names)
print(img_paths)
print(gender)
df = pd.DataFrame()
df['IMAGE PATH'] = img_paths
df['IMAGE NAME'] = img_names
df['LABEL'] = labels
df['GENDER'] = gender

In [ ]:
#check if data already rebalance one to each others (optional)
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
sns.countplot(ax=axes[0], data = df, x = 'LABEL')
sns.countplot(ax=axes[1], data = df, x = 'LABEL', hue = 'GENDER')

In [ ]:
df.drop(columns = 'GENDER',inplace=True) #drop table gender
df.head()

In [ ]:
#rename (optional)
df['MAPPED LABELS'] = [map_classes[i] for i in df['LABEL']]
df = df.sample(frac = 1)
df.to_csv('dataset.csv')
df.head()

In [ ]:
#show data (optional)
dim = len(classes)
fig,axes = plt.subplots(1,dim)
fig.subplots_adjust(0,0,2,2)
for idx, i in enumerate(classes):
    dum = df[df['LABEL'] == i]
    random_num = random.choice(dum.index)
    label = df.loc[random_num]['LABEL']
    axes[idx].imshow(cv2.imread(df.loc[random_num]['IMAGE PATH']))
    axes[idx].set_title("CLASS: "+label +"\n" +  "LABEL:"+str(map_classes[label]))
    axes[idx].axis('off')

In [ ]:
import fingerprint_feature_extractor

X_data = df['IMAGE PATH']
y_data = df['MAPPED LABELS']
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, shuffle=True, test_size=0.2,stratify=y_data)
#Creating numpy arrays of images
X = []
y = []
for img in X_train:
    #FeaturesTerminations, FeaturesBifurcations = fingerprint_feature_extractor.extract_minutiae_features(img, spuriousMinutiaeThresh=10, invertImage=False, showResult=True, saveResult=True)
    #feature_concatenate = np.concatenate((FeaturesTerminations, FeaturesBifurcations))
    #print(feature_concatenate.flatten())
    #X.append(feature_concatenate.flatten())
    X.append(cv2.imread(img))
    print(cv2.imread(img))
for i in y_train:
    y.append(i)
X = np.array(X)
y = np.array(y)

# Converting the labels vector to one-hot format
y = keras.utils.to_categorical(y, 5)

In [ ]:
print(f"Total number of Images: {len(X_data)}")
print(f"Number of Training Images: {len(X_train)}")
print(f"Number of Test Images: {len(X_test)}") # Saving a small number of images for model testing|
print(f"Shape of Images: {X[0].shape}") #Printing the shape of Images

In [ ]:
model = keras.Sequential(
    [
        layers.Conv2D(32, input_shape=(128,128,3),padding="same",kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(32, kernel_size=(3, 3), padding="same",activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), padding="same",activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3),padding="same",activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3),padding="same",activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(128, kernel_size=(3, 3),padding="same",activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(5, activation="softmax",kernel_regularizer='l1_l2'),
    ]
)

In [ ]:
#configure hyperparameters
LOSS = 'categorical_crossentropy'
BATCH_SIZE = 64
EPOCHS = 20
LEARNING_RATE = 0.002
OPTIMIZER = keras.optimizers.Adam(learning_rate=LEARNING_RATE)

In [ ]:
model.compile(loss=LOSS, optimizer=OPTIMIZER, metrics=['accuracy'])

history=model.fit(x=X, y=y, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.2)

In [ ]:
#Saving the model
model.save('enhancedfingerprint.h5')

In [ ]:
#Optional
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='center right')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

In [ ]:
#convert testing data
test_X = []
for i in X_test:
    im = cv2.imread(i)
    print(im)
    im = np.reshape(im, (1,128,128,3))
    test_X.append(im)
test_X = np.array(test_X)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import random
import matplotlib.pyplot as plt

fig, axes = plt.subplots(5, 5)
fig.subplots_adjust(0, 0, 3, 3)

f1_scores = []
precision_scores = []
accuracy_scores = []
recall_scores = []

for i in range(5):
    for j in range(5):
        num = random.randint(0, len(test_X) - 1)
        display_image = test_X[num].squeeze(0)
        image = test_X[num]
        predicted_prob = model.predict(image)
        predicted_class = np.argmax(predicted_prob)
        ground_truth = classes[y_test.iloc[num]]
        axes[i, j].imshow(display_image)
        if classes[predicted_class] != classes[y_test.iloc[num]]:
            t = 'PREDICTED {} \n GROUND TRUTH[{}]'.format(classes[predicted_class], classes[y_test.iloc[num]])
            axes[i, j].set_title(t, fontdict={'color': 'darkred'})
        else:
            t = '[CORRECT] {}'.format(classes[predicted_class])
            axes[i, j].set_title(t)
        axes[i, j].axis('off')

        # Calculate evaluation metrics
        predicted_labels = np.array([classes[predicted_class]])
        ground_truth_labels = np.array([ground_truth])
        f1_scores.append(f1_score(ground_truth_labels, predicted_labels, average='macro'))
        precision_scores.append(precision_score(ground_truth_labels, predicted_labels, average='macro'))
        accuracy_scores.append(accuracy_score(ground_truth_labels, predicted_labels))
        recall_scores.append(recall_score(ground_truth_labels, predicted_labels, average='macro'))

plt.show()

# Print average evaluation scores
print("Average F1 Score:", np.mean(f1_scores))
print("Average Precision:", np.mean(precision_scores))
print("Average Accuracy:", np.mean(accuracy_scores))
print("Average Recall:", np.mean(recall_scores))
